In [ ]:
# Processor requirements
# The first two cells must be code cells, inpath and outpath define the scan file location and output location
inpath = "/dls/science/groups/das/ExampleData/hdfmap_tests/i16/1109527.nxs"
outpath = "output.nxs"
nxs2dat = "false"

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex

import nexus2srs
from mmg_toolbox import data_file_reader, module_info
from mmg_toolbox.plotting.matplotlib import set_plot_defaults

print(module_info())
set_plot_defaults()  # set custom matplotlib rcParams

def md(string):
    return display(Markdown(string))

In [ ]:
md('# 1. Convert NeXus file to .dat file')

dat_folder = os.path.join(os.path.dirname(inpath), 'spool')
if nxs2dat == 'true' or nxs2dat == True:
    nexus2srs.run_nexus2srs(inpath, dat_folder, '--tiff', '--info')

In [ ]:
md('# 2. Read Metadata from NeXus file')
scan = data_file_reader(inpath)

In [ ]:
try:
    md(scan.format('# {beamline} Scan: {scan_number:.0f}\n*{filepath}*'))

    # Time
    start, stop = scan.times('start_time', 'end_time')
    duration = stop - start

    # Create table
    tab = '| field | metadata |\n| --- | --- |\n'
    for s in str(scan).split('\n'):
        if '=' in s:
            tab += '| %s |\n' % s.replace('=', '|')

    tab += '|**start time** | %s |\n' % start.strftime('%Y-%m-%d %H:%M')
    tab += '|**end time** | %s |\n' % stop.strftime('%Y-%m-%d %H:%M')
    tab += '|**duration** | %s |\n' % duration

    md(tab)
except Exception as e:
    print('Exception occured:', e, '\n')
    print(scan)

In [ ]:
md('# 3. Default Plot')
ax = scan.plot()

In [ ]:
md('# 4. Example Detector Image')
if scan.map.image_data:
    detector = next(iter(scan.map.image_data))
    md(f'# {detector} Image')
    ax = scan.plot.image()
else:
    print('No image file found')

In [ ]:
md('# 5. Scannable Data')
scannables = scan.map.get_scannables(scan.load_hdf(), flatten=True)

table = '| ' + '|'.join(scannables) + ' |\n| ' + ' --- | ' * len(scannables) + '\n'
for n in range(scan.map.scannables_length()):
    table += '| ' + '|'.join(str(array[n]) for array in scannables.values()) + ' |\n'

md(table)

In [ ]:
from hdfmap import hdf_tree_string
md('# 6. Nexus Structure')
print(hdf_tree_string(inpath))